### InternVL2_5-4B-MPO

### 모델, 토크나이저, 질의 데이터 초기화

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

# If you want to load a model using multiple GPUs, please refer to the `Multiple GPUs` section.
model_id = 'OpenGVLab/InternVL2_5-4B-MPO'
model = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import os
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

dataset_name = 'KoAlpaca_v1.1.jsonl'
data_files = {"total": dataset_name}

dataset = load_dataset(path=os.getcwd(), data_files=data_files)
total_data = dataset['total']
train_dataset, eval_dataset = total_data.train_test_split(train_size=0.05, seed=42).values()
total_data = train_dataset.remove_columns('url')

def Tokenize_function(example):
    return tokenizer(example['instruction'], example['output'], truncation=True)

tokenized_data = total_data.map(Tokenize_function, batched=True)
training_args = TrainingArguments("test-trainer")

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_data,
    data_collator=data_collator
)
trainer.train()

### 질의 데이터 가져오기

In [ ]:
from PIL import Image

english_1_url = "english_1_a_1.png"
english_2_url = "english_2_a_4.png"
english_3_url = "english_3_a_2.png"

math_1_url = "math_1_a_1.png" 
math_2_url = "math_2_a_5.png"
math_3_url = "math_3_a_5.png"

korean_1_url = "korean_1_a_1.png" 
korean_2_url = "korean_2_a_1.png"
korean_3_url = "korean_3_a_2.png"

english_1_url_image = Image.open(english_1_url)
english_2_url_image = Image.open(english_2_url)
english_3_url_image = Image.open(english_3_url)

math_1_url_image = Image.open(math_1_url)
math_2_url_image = Image.open(math_2_url)
math_3_url_image = Image.open(math_3_url)

korean_1_url_image = Image.open(korean_1_url)
korean_2_url_image = Image.open(korean_2_url)
korean_3_url_image = Image.open(korean_3_url)

### 영어 질의 하기

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(english_1_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)

# single-image multi-round conversation
question = '<image>\nExtract words'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(english_1_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=2048, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(english_2_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=2048, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(english_3_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=2048, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

### 수학 질의 하기

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(math_1_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=2048, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(math_2_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(math_3_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

### 국어 질의 하기

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(korean_3_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)

# single-image multi-round conversation
question = '<image>\nExtract words'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(korean_1_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=2048, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(korean_2_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=2048, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")

In [ ]:
# 이미지 텍스트 추출
# 동작 시간 측정 
import time

start = time.time()

# set the max number of tiles in `max_num`
pixel_values = load_image(korean_3_url, max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=2048, do_sample=True)

# single-image multi-round conversation
question = '<image>\nProvide an explanation of the correct answer to the question. Please speak korean.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

end = time.time()

print(f"{end - start:.5f} sec")